# Projeto de Classificação de Texto - IMDB (Pt-Br)

**LinkedIn**: https://linkedin.com/in/matheusjerico <br>
**Github**: https://github.com/matheusjerico

#### O que será feito?

- Ler o conjunto de dados que o Luís Fred forneceu na Plataforma do Kaggle (https://www.kaggle.com/luisfredgs/imdb-ptbr/)
- Transformar os textos das avaliação em valores numéricos usando Pipeline do Scikit-learn
- Criar um Classificador, treinar e realizar predições
- Testar o Classificador com novas avaliações
- Avaliar o desempenho do Classificador

#### Qual o objetivo?
Realizar a Análise de Sentimento do conjunto de Dados de Avaliações de filmes do IMDB, em português. Construir um Classificador com o objetivo de classificar se a avaliação foi **Positiva** ou **Negativa**.

## 1. Importar bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import stem


## 2. Carregar conjunto de dados

- unzip arquivo

In [ ]:
# ! unzip -n ../dataset/imdb-ptbr.zip  -d ../dataset
# ! ls ../dataset/

- carregando conjutno de dados

In [ ]:
dataset = pd.read_csv("../input/imdb-ptbr/imdb-reviews-pt-br.csv")

In [ ]:
dataset.drop(columns=['text_en', 'id'], inplace=True)

- primeiros registros do dataset

In [ ]:
dataset.head()

- últimos registros do dataset

In [ ]:
dataset.tail()

- dimensão do dataset

In [ ]:
dataset.shape

- Vamos visualizar um registro negativo:

In [ ]:
display(Markdown('> '+dataset['text_pt'][2]))

## 2. Análise Exploratória

### 2.1. Valores NaN

In [ ]:
dataset.isnull().sum()

### 2.2. Avaliações vazias?

In [ ]:
blk = []

for index, text, label in dataset.itertuples():
    if type(text) == str:
        if text.isspace():
            blk.append(i)

print(f"{len(blk)}, vazios: {blk}")

### 2.3. Distribuição das avaliações

In [ ]:
dataset['sentiment'].value_counts()

- Dataset é balanceado.

## 3. Processamento dos Dados

In [ ]:
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z]')
STOPWORDS = set(stopwords.words('portuguese'))
stemmer = stem.RSLPStemmer()

In [ ]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError):  # unicode is a default on python 3
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")

    return str(text)

def preprocess_stemming_text(text):
    text = text.lower()

    text = strip_accents(text)
    text = BAD_SYMBOLS_RE.sub(' ', text)
    text = ' '.join(stemmer.stem(word)
                    for word in text.split() if word not in STOPWORDS)

    return text 

In [ ]:
dataset['text_pt_stemm'] = dataset['text_pt'].apply(preprocess_stemming_text)
dataset.head()

## 4. Divisão dos Dados

- dividindo os dados em treino e teste

In [ ]:
X = dataset['text_pt_stemm']
y = dataset['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

## 5. Construindo Pipeline 
- 1º etapa: TD-IDF
- 2º etapa: Machine learning

### 5.1. Naive Bayes

In [ ]:
clf_nb = Pipeline([('TF-IDF', TfidfVectorizer(lowercase=True, 
                                              strip_accents='unicode',
                                              stop_words=stopwords.words('portuguese'))),
                   ('Classificador', MultinomialNB()),
])

### 5.2. Linear SVC

In [ ]:
clf_svc = Pipeline([('TF-IDF', TfidfVectorizer(lowercase=True, 
                                               strip_accents='unicode',
                                               stop_words=stopwords.words('portuguese'))),
                   ('Classificador', LinearSVC()),
])

### 5.3. Random Forest

In [ ]:
clf_rf = Pipeline([('TF-IDF', TfidfVectorizer(lowercase=True, 
                                              strip_accents='unicode', 
                                              stop_words=stopwords.words('portuguese'))),
                   ('Classificador', RandomForestClassifier()),
])

## 6. Treinamento, Predição e Avaliação - ML

### 6.1. Naive Bayes

- Treinamento

In [ ]:
clf_nb.fit(X_train, y_train)

- Predição

In [ ]:
preds = clf_nb.predict(X_test)

- Matriz de Confusão

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.heatmap(confusion_matrix(y_test, preds), annot=True, fmt="d");
ax.set_title("Matriz de Confusão - Naive Bayes", fontsize=20)
ax.set_ylabel('Classe Verdadeira', fontsize=15)
ax.set_xlabel('Classe Predita', fontsize=15)

- Precisão, Revocação e F1-Score

In [ ]:
print(classification_report(y_test,preds))

- Acurácia

In [ ]:
print(accuracy_score(y_test,preds))

### 6.2. Linear SVC

- treinamento

In [ ]:
clf_svc.fit(X_train, y_train)

- predição

In [ ]:
preds = clf_svc.predict(X_test)

- matriz de confusão

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.heatmap(confusion_matrix(y_test, preds), annot=True, fmt="d");
ax.set_title("Matriz de Confusão - Linear SVC", fontsize=20)
ax.set_ylabel('Classe Verdadeira', fontsize=15)
ax.set_xlabel('Classe Predita', fontsize=15)

- Precisão, Revocação e F1-Score

In [ ]:
print(classification_report(y_test,preds))

- acurácia

In [ ]:
print(accuracy_score(y_test,preds))

### 6.3. Random Forest

- treinamento

In [ ]:
clf_rf.fit(X_train, y_train)

- predição

In [ ]:
preds = clf_rf.predict(X_test)

- matriz de confusão

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.heatmap(confusion_matrix(y_test, preds), annot=True, fmt="d");
ax.set_title("Matriz de Confusão - Random Forest", fontsize=20)
ax.set_ylabel('Classe Verdadeira', fontsize=15)
ax.set_xlabel('Classe Predita', fontsize=15)

- acurácia

In [ ]:
print(accuracy_score(y_test,preds))

## 7. Análise

O algorítmo que obteve o melhor desempenho foi o Linear SVC.
- Acurácia: 89%
- Revocação: 90%
- F1-Score: 89%

## 8. Predição de nova avaliação

In [ ]:
avaliacao_supernatural = "A série Supernatural é longa, mas é muito boa e emocionante. \
Você se apega aos personagens, sofre por eles.. É uma série cheia de altos e baixos."

In [ ]:
avaliacao_purge = "A série é chata, o enredo é péssimo, horrível, Não gostei da śerie"

In [ ]:
print(clf_svc.predict([avaliacao_supernatural]))

In [ ]:
print(clf_svc.predict([avaliacao_purge]))

## FIM